## Import packages

In [6]:
import torch
import time
import cortex
import numpy as np
import nibabel as nib
import matplotlib as mpl
import matplotlib.pyplot as plt

from matplotlib import gridspec
from Gallant_lab_func.utils import generate_leave_one_run_out
from Gallant_lab_func.delayer import Delayer
from sklearn import set_config
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import check_cv
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from scipy.ndimage import gaussian_filter1d
from scipy.stats import zscore, pearsonr, ttest_1samp, spearmanr, kendalltau, ttest_rel
from nltools.stats import isc, fdr, fisher_r_to_z
from nltools.external import hrf
from nilearn.glm import threshold_stats_img
from statsmodels.stats.multitest import fdrcorrection
from himalaya.kernel_ridge import KernelRidgeCV, MultipleKernelRidgeCV, ColumnKernelizer, Kernelizer
from himalaya.viz import plot_alphas_diagnostic
from himalaya.backend import set_backend
from himalaya.ridge import RidgeCV, GroupRidgeCV, ColumnTransformerNoStack
from himalaya.scoring import r2_score_split

result_folder = '/media/dasom/caee0336-77a9-438c-af2f-7cf5b88293e7/dasom/SM_DATA/social_movie1/results'

mpl.rcParams['axes.linewidth'] = 2
mpl.rcParams['xtick.major.width'] = 2
mpl.rcParams['ytick.major.width'] = 2
mpl.rcParams['xtick.major.size'] = 5
mpl.rcParams['ytick.major.size'] = 5
mpl.rcParams['axes.spines.top'] = True
mpl.rcParams['axes.spines.right'] = True
mpl.rcParams['axes.spines.bottom'] = True
mpl.rcParams['axes.spines.left'] = True
np.set_printoptions(precision=6, suppress=True)

## Load all data

In [7]:
excluded_tr_ep1 = np.concatenate([np.arange(10)])
excluded_tr_ep2 = np.concatenate([np.arange(44)])
excluded_tr_ep3 = np.concatenate([np.arange(44)])

rest_tr_ep1 = np.delete(np.arange(996), excluded_tr_ep1)
rest_tr_ep2 = np.delete(np.arange(958), excluded_tr_ep2)
rest_tr_ep3 = np.delete(np.arange(710), excluded_tr_ep3)
rest_total = rest_tr_ep1.shape[0] + rest_tr_ep2.shape[0] + rest_tr_ep3.shape[0]
rest_tr = np.hstack([rest_tr_ep1, rest_tr_ep2+996, rest_tr_ep3+996+958])
print(f'{rest_tr_ep1.shape[0]} + {rest_tr_ep2.shape[0]} + {rest_tr_ep3.shape[0]} = {rest_total}')

986 + 914 + 666 = 2566


In [10]:
# moten_features = []
# for r in range(3):
#     moten_feature = np.load(f'/home/dasom/SM_DATA/videodata/social_movie1/moten_features/moten_run_{r+1}.npy')
#     moten_features.append(moten_feature)
# moten_features = np.vstack([moten_features[r] for r in range(3)])

features0 = np.load(f'{result_folder}/MODEL/TR_full.npy', allow_pickle = True)
features1 = np.load(f'{result_folder}/MODEL/TR_cooccur.npy', allow_pickle = True)
features2 = np.load(f'{result_folder}/MODEL/TR_senti_pos.npy', allow_pickle = True)
features3 = np.load(f'{result_folder}/MODEL/TR_senti_neg.npy', allow_pickle = True)

features1_gf5 = gaussian_filter1d(features1, 5, axis=0)
features2_gf5 = gaussian_filter1d(features2, 5, axis=0)
features3_gf5 = gaussian_filter1d(features3, 5, axis=0)

features = np.hstack([features1, features2, features3])
features = features[rest_tr,:]

social_movie_run = np.array([493,493,457,457,333,333]) # 6 folds
onsets = np.cumsum(social_movie_run)
features_split = np.split(features, onsets, axis=0)

brain_data_sc_sm = np.load(f'{result_folder}/fMRI_data/movie_data_ex1_sc_sm.npy', allow_pickle = True)[:,:,rest_tr]

In [11]:
print('fMRI data shape : ', brain_data_sc_sm.shape)
print('feature shape : ', features.shape)

fMRI data shape :  (24, 69831, 2566)
feature shape :  (2566, 90)


## Define and Run banded ridge regression model

### a single feature space

In [ ]:
set_config(display='diagram', assume_finite=True)
backend = set_backend("torch_cuda", on_error="warn") # torch cuda setting

torch.manual_seed(2022)
np.random.seed(2022) # random seed setting

alphas = np.logspace(0, 20, 41) # ridge hyperparameter range

n_targets_batch = 10000
n_alphas_batch = 5
n_targets_batch_refit = 100

sub_r_scores = []
sub_r2_scores = []
weights = []
for sub in [3]:
    r_scores_cv = []
    r2_scores_cv = []
    weight = []
    brain_split = np.split(brain_data_sc_sm[sub], onsets, axis=1) # 6 folds
    for train_idx, test_idx in LeaveOneOut().split(range(len(social_movie_run))): # generating and runing 6 folds

        train_run_onsets = np.cumsum([0]+social_movie_run[train_idx].tolist())[:-1]

        X_train = np.nan_to_num(np.vstack(([zscore(features_split[i], axis=0, nan_policy='omit') for i in train_idx])))
        X_train = np.vstack([np.convolve(X_train[:,i],hrf.glover_hrf(1,1)) for i in range(X_train.shape[1])])[:,:X_train.shape[0]].T # check HRF convolve
        X_test = np.nan_to_num(zscore(features_split[test_idx[0]], axis=0, nan_policy='omit'))
        X_test = np.vstack([np.convolve(X_test[:,i],hrf.glover_hrf(1,1)) for i in range(X_test.shape[1])])[:,:X_test.shape[0]].T # check shape

        y_train = np.hstack(([zscore(brain_split[i], axis=1, nan_policy='omit') for i in train_idx])).T
        y_test = zscore(brain_split[test_idx[0]], axis=1, nan_policy='omit').T # check shape
        
        X_train = X_train.astype("float32") # reducing computing cost
        X_test = X_test.astype("float32")
        y_train = y_train.astype("float32")
        y_test = y_test.astype("float32")

        n_samples_train = X_train.shape[0]
        cv = generate_leave_one_run_out(n_samples_train, train_run_onsets) # nested cross-validation for obtaining optimal alpha
        cv = check_cv(cv)

        pipeline = make_pipeline(
            StandardScaler(with_mean=True, with_std=False), # z-scoring
            Delayer(delays=[0]), # [0,1,2,3,4,5,6,7]
            RidgeCV(
                Y_in_cpu=True,
                alphas=alphas, 
                cv=cv,
                solver_params=dict(n_targets_batch=n_targets_batch, 
                                   n_alphas_batch=n_alphas_batch,
                                   n_targets_batch_refit=n_targets_batch_refit)),
            verbose=True
        )

        pipeline.fit(X_train, y_train)
        pred_y = pipeline.predict(X_test)
        r = np.array([fisher_r_to_z(pearsonr(y_test[:,r], pred_y[:,r])[0]) for r in range(y_train.shape[1])]) # pearson correlation
        r2 = pipeline.score(X_test, y_test) # r-squared value
        r2 = backend.to_numpy(r2)

        r_scores_cv.append(r)
        r2_scores_cv.append(r2)
        weight.append(backend.to_numpy(pipeline.named_steps['ridgecv'].coef_)) # weight for each feature
        torch.cuda.empty_cache() # release reusable GPU memory

    sub_r_scores.append(np.nanmean(r_scores_cv, axis = 0)) # average model prediction scores across folds
    sub_r2_scores.append(np.nanmean(r2_scores_cv, axis = 0))
    weights.append(weight)

sub_r_scores = np.array(sub_r_scores)
sub_r2_scores = np.array(sub_r2_scores)
weights = np.array(weights)

In [16]:
print('weights : ', weights.shape)
print('r score : ', sub_r_scores.shape)
print('r2 score : ', sub_r2_scores.shape)

weights :  (1, 6, 90, 69831)
r score :  (1, 69831)
r2 score :  (1, 69831)


### multiple feature spaces

In [43]:
start = time.time()
set_config(display='diagram', assume_finite=True)
backend = set_backend("torch_cuda", on_error="warn")

torch.manual_seed(2022)
np.random.seed(2022) # random seed setting

feature_names = ["co-occur", "valence"] # feature subspaces

n_iter = 20
alphas = np.logspace(0, 20, 41)

n_targets_batch = 10000 # 200
n_alphas_batch = 5
n_targets_batch_refit = 100 # 200

r_scores_cv0 = []
r_scores_cv1 = []
r_scores_cv2 = []
r2_scores_cv0 = []
r2_scores_cv1 = []
r2_scores_cv2 = []
weights = []
for sub in [3]:
    r_scores_batch0 = []
    r_scores_batch1 = []
    r_scores_batch2 = []
    r2_scores_batch0 = []
    r2_scores_batch1 = []
    r2_scores_batch2 = []
    weight = []

    brain_split = np.split(brain_data_sc_sm[sub], onsets, axis=1)

    for train_idx, test_idx in LeaveOneOut().split(range(len(social_movie_run))):
        train_run_onsets = np.cumsum([0]+social_movie_run[train_idx].tolist())[:-1]
        X_train = np.nan_to_num(np.vstack(([zscore(features_split[i], axis=0, nan_policy='omit') for i in train_idx])))
        X_train = np.vstack([np.convolve(X_train[:,i],hrf.glover_hrf(1,1)) for i in range(X_train.shape[1])])[:,:X_train.shape[0]].T
        X_test = np.nan_to_num(zscore(features_split[test_idx[0]], axis=0, nan_policy='omit'))
        X_test = np.vstack([np.convolve(X_test[:,i],hrf.glover_hrf(1,1)) for i in range(X_test.shape[1])])[:,:X_test.shape[0]].T

        y_train = np.hstack(([zscore(brain_split[i], axis=1, nan_policy='omit') for i in train_idx])).T
        y_test = zscore(brain_split[test_idx[0]], axis=1, nan_policy='omit').T
        
        X_train = X_train.astype("float32")
        X_test = X_test.astype("float32")
        y_train = y_train.astype("float32")
        y_test = y_test.astype("float32")
        
        n_features_list = [30, 60] # dividing into two feature subspaces
        n_features_list_cumsum = np.cumsum([0] + n_features_list)
        
        solver = "random_search" # algorithm for optimizing the log group scalings deltas over cross-validation

        solver_function = GroupRidgeCV.ALL_SOLVERS[solver]

        n_samples_train = X_train.shape[0]
        cv = generate_leave_one_run_out(n_samples_train, train_run_onsets)
        cv = check_cv(cv)

        solver_params = dict(n_iter=n_iter, 
                             alphas=alphas,
                             n_targets_batch=n_targets_batch,
                             n_alphas_batch=n_alphas_batch,
                             n_targets_batch_refit=n_targets_batch_refit)
        
        mkr_model = GroupRidgeCV(groups="input", solver=solver,
                                 solver_params=solver_params, cv=cv) 

        set_config(display='diagram')

        preprocess_pipeline = make_pipeline(
            StandardScaler(with_mean=True, with_std=False),
            Delayer(delays=[0]),
            )

        # Find the start and end of each feature space in the concatenated X_train.
        start_and_end = np.concatenate([[0], np.cumsum(n_features_list)])
        slices = [
            slice(start, end)
            for start, end in zip(start_and_end[:-1], start_and_end[1:])
        ]
        slices

        kernelizers_tuples = [(name, preprocess_pipeline, slice_)
                              for name, slice_ in zip(feature_names, slices)] # for each feature subspace
        
        column_kernelizer = ColumnTransformerNoStack(kernelizers_tuples)
        column_kernelizer

        pipeline = make_pipeline(
                        column_kernelizer,
                        mkr_model,
                        verbose=False
                        )

        pipeline.fit(X_train, y_train)

        scores = pipeline.score(X_test, y_test)
        r2_scores_batch0.append(backend.to_numpy(scores))

        y_test_pred_split = pipeline.predict(X_test, split=True) # get prediction scores for each feature subspace
        split_scores = r2_score_split(y_test, y_test_pred_split)  
        r2_scores_batch1.append(backend.to_numpy(split_scores[0]))
        r2_scores_batch2.append(backend.to_numpy(split_scores[1]))
        
        pred_y = pipeline.predict(X_test)
        pred_y = backend.to_numpy(pred_y)
        r_scores_batch0.append(np.array([fisher_r_to_z(pearsonr(y_test[:,r], pred_y[:,r])[0]) for r in range(y_train.shape[1])]))

        pred_y1 = y_test_pred_split[0]
        pred_y1 = backend.to_numpy(pred_y1)
        r_scores_batch1.append(np.array([fisher_r_to_z(pearsonr(y_test[:,r], pred_y1[:,r])[0]) for r in range(y_train.shape[1])]))

        pred_y2 = y_test_pred_split[1]
        pred_y2 = backend.to_numpy(pred_y2)
        r_scores_batch2.append(np.array([fisher_r_to_z(pearsonr(y_test[:,r], pred_y2[:,r])[0]) for r in range(y_train.shape[1])]))
        
        weight.append(backend.to_numpy(pipeline.named_steps['groupridgecv'].coef_))
        torch.cuda.empty_cache()
        
    r_scores_cv0.append(np.nanmean(r_scores_batch0, axis=0))
    r_scores_cv1.append(np.nanmean(r_scores_batch1, axis=0))
    r_scores_cv2.append(np.nanmean(r_scores_batch2, axis=0))
    r2_scores_cv0.append(np.nanmean(r2_scores_batch0, axis=0))
    r2_scores_cv1.append(np.nanmean(r2_scores_batch1, axis=0))
    r2_scores_cv2.append(np.nanmean(r2_scores_batch2, axis=0))
    weights.append(weight)

r_scores_cv0 = np.array(r_scores_cv0)
r_scores_cv1 = np.array(r_scores_cv1)
r_scores_cv2 = np.array(r_scores_cv2)
r2_scores_cv0 = np.array(r2_scores_cv0)
r2_scores_cv1 = np.array(r2_scores_cv1)
r2_scores_cv2 = np.array(r2_scores_cv2)
weights = np.array(weights)
weights_mean = np.mean(weights, axis=1)

[........................................] 100% | 132.24 sec | 100 random sampling with cv | 
[........................................] 100% | 131.45 sec | 100 random sampling with cv | 
[........................................] 100% | 134.13 sec | 100 random sampling with cv | 
[........................................] 100% | 134.06 sec | 100 random sampling with cv | 
[........................................] 100% | 144.00 sec | 100 random sampling with cv | 
[........................................] 100% | 143.04 sec | 100 random sampling with cv | 


### Computing p-values

#### Huth et al., 2016 // Statistical test for each subject separately

In [19]:
import statsmodels.regression.linear_model as sm

np.random.seed(2022)

social_movie_run = np.array([493,493,457,457,333,333])
TR_length = np.array([np.sum(social_movie_run[np.delete(np.arange(6), r)]) for r in range(6)])
r2_dist = np.sort([np.mean([sm.OLS(zscore(np.random.normal(size = TR_length[r])), zscore(np.random.normal(size = TR_length[r]))).fit().rsquared for r in range(6)]) for rr in range(1000)])
r_dist = np.sort([np.mean([pearsonr(np.random.normal(size = TR_length[r]), np.random.normal(size = TR_length[r]))[0] for r in range(6)]) for rr in range(1000)])

In [34]:
p_r2 = np.array([np.sum(sub_r2_scores[0][v] < r2_dist)/1000 for v in range(69831)])
p_r = np.array([np.sum(sub_r_scores[0][v] < r_dist)/1000 for v in range(69831)])

In [37]:
print('fdr-corrected p_r2 : ', fdr(p_r2))
print('fdr-corrected p_r : ', fdr(p_r))

fdr-corrected p_r2 :  0.005
fdr-corrected p_r :  0.024


#### Kumar et al., 2024 // Group statistics

In [717]:
# One-sample bootstrap hypothesis test
def work_func2(voxels):
    np.random.seed(2022)

    # r_scores_cv0 = np.load(f'{result_folder}/ENCODING/r_value_cv_full_2fs_f90_score0_gf5.npy')

    repetition = 5000
    rand_dist = []
    for v in voxels:
        ran_score = []
        for r in range(repetition):
            ran_score.append(np.mean(np.random.choice(r_scores_cv0[:,v], size=24, replace=True)))
        rand_dist.append(ran_score)
    return rand_dist

In [718]:
from multiprocessing import Pool
import time

np.random.seed(2022)

cpu = 10
pool = Pool(cpu)

start = int(time.time())
results = pool.map(work_func2, np.array_split(np.arange(69831), cpu))
rand_dist = np.concatenate([results[r] for r in range(cpu)])
print("***run time(sec) :", int(time.time()) - start)

p0 = np.array([np.sum(rand_dist[:,v] < 0)/5000 for v in range(69831)])
th_p0 = fdr(p0)

***run time(sec) : 1402


## Visualization

In [ ]:
cc = nib.load('./files/MNI152NLin2009cAsym_3mm_mask.nii.gz')
brain_mask = nib.load('./files/MNI152NLin2009cAsym_3mm_mask.nii.gz').get_fdata()
score_brain = np.zeros((brain_mask.shape))
score_brain[np.where(brain_mask > 0)] = t3
z_map = nib.Nifti1Image(score_brain, cc.affine, cc.header)

clean_map, threshold = threshold_stats_img(z_map, alpha=.05, height_control='fdr', cluster_threshold=0)
clean_map_data = clean_map.get_fdata()
clean_map_data[clean_map_data <= 0] = 0
print(threshold)

clean_map_data3 = clean_map_data

### Single space

In [54]:
roi_mask = nib.load('./files/BNA_3mm_atlas.nii.gz').get_fdata()
brain_mask = nib.load('./files/MNI152NLin2009cAsym_3mm_mask.nii.gz').get_fdata()

subj = 0
visualized_data = sub_r_scores[subj].copy()
visualized_data[p_r > fdr(p_r)] = np.nan
pycor_mask = np.full(brain_mask.shape, np.nan)
pycor_mask[brain_mask > 0] = visualized_data
    
vol_data = cortex.Volume(pycor_mask.transpose(2,1,0), 'cvs_avg35_inMNI152', 'social_movie', cmap='plasma',
                        vmin = 0, vmax = 0.03)
cortex.webshow(vol_data)

Started server on port 14155


<JS: window.viewer>

Stopping server
Stopping server


### Multiple spaces

In [52]:
brain_mask = nib.load('./files/MNI152NLin2009cAsym_3mm_mask.nii.gz').get_fdata()

scores_cv1 = np.load(f'{result_folder}/ENCODING/r_value_cv_full_2fs_f90_score1_gf5.npy')
scores_cv2 = np.load(f'{result_folder}/ENCODING/r_value_cv_full_2fs_f90_score2_gf5.npy')

# visualized_data1 = scores_cv1[3].copy()
# visualized_data1[visualized_data1 < 0] = 0
# visualized_data2 = scores_cv2[3].copy()
# visualized_data2[visualized_data2 < 0] = 0

score_brain1 = np.zeros(brain_mask.shape)
score_brain1[brain_mask==1] = visualized_data2

score_brain2 = np.zeros(brain_mask.shape)
score_brain2[brain_mask==1] = visualized_data1


vol_data = cortex.Volume2D(score_brain1.transpose(2,1,0), score_brain2.transpose(2,1,0), "cvs_avg35_inMNI152", "social_movie",
                           vmin = 0, vmax = 0.05, vmin2 = 0, vmax2 = 0.05,
                           cmap='MaWtCy_covar')
                           # cmap='RdYlBu_covar')
cortex.webshow(vol_data)

Started server on port 39755


<JS: window.viewer>

Stopping server
